# Interacting with Assets in VAM API

This notebook provides a comprehensive guide on how to interact with different types of assets using the VAM API.
We will cover:
- Querying different asset types
- Filtering assets by execution venue
- Working with different asset types like spot, futures, and equity
- Performing batch operations



We will demonstrate how to query assets across these execution venues.


In [1]:
import os
from pathlib import Path

# Save the original working directory (only once)
try:
    original_wd
except NameError:
    original_wd = os.getcwd()

# Compute the target directory: two levels up from the original working directory
# For example, if original_wd is /Users/username/project/notebooks,
# then target_dir becomes /Users/username
target_dir = Path(original_wd).parents[2]

# Change the working directory to the target directory
os.chdir(target_dir)
print("Working directory set to:", os.getcwd())

Working directory set to: /home/jose/code/MainSequenceClientSide/mainsequence-sdk


In [2]:

import dotenv
dotenv.load_dotenv('.env')

import mainsequence 
from mainsequence.client import Asset, AssetFutureUSDM, AssetCurrencyPair,AssetCategory
from mainsequence.client import MARKETS_CONSTANTS

# Define execution venue symbols
BINANCE_EV = MARKETS_CONSTANTS.BINANCE_EV_SYMBOL
BINANCE_FUTURES_EV = MARKETS_CONSTANTS.BINANCE_FUTURES_EV_SYMBOL
ALPACA_EV = MARKETS_CONSTANTS.ALPACA_EV_SYMBOL

## Optimal Asset Queries

There are many assets, and there will always be more. We recommend narrowing your search so that the expected results are in the hundreds. If you need more assets, it is best to first create a category that encompasses those assets and then query by that category.

A good way to start narrowing asset searches is by filtering on the execution venue and some FIGI properties. For example, if you want to retrieve the equivalent assets from one category on another exchange, you could do something like this:



In [3]:
top_100_cryptos = AssetCategory.get(unique_identifier="top_100_crypto_market_cap")

# Switch to Binance categories
spot_assets = Asset.filter(id__in=top_100_cryptos.assets)

# Get them through the main sequence FIGI class and exchange
binance_currency_pairs = AssetCurrencyPair.filter(
    base_asset__figi_details__main_sequence_share_class__in=[
        a.figi_details.main_sequence_share_class for a in spot_assets
    ],
    execution_venue__symbol=MARKETS_CONSTANTS.BINANCE_EV_SYMBOL,
    quote_asset__symbol="USDT",
    include_base_quote_detail=False
)



## Filtering Assets by Type

Different asset types have distinct properties, which can help you extend your filters. It's important to note that all asset types inherit from the Asset class. Therefore, you can always use the Asset class to access all assets and their general properties.

In [4]:
# Filtering Simple Assets
simple_assets = Asset.filter(symbol="BTCUSDT")
print(f"Total Spot Crypto Assets on Binance: {len(simple_assets)}")

# Filtering Futures
cash_equity_assets = AssetCurrencyPair.filter(symbol="BTCUSDT")
print(f"Total Assets CurrencyPair Assets : {len(cash_equity_assets)}")

# Filtering  Futures USDM Assets
futures_assets= AssetFutureUSDM.filter(symbol="BTCUSDT")
print(f"Total Futures Assets : {len(futures_assets)}")




Total Spot Crypto Assets on Binance: 5
Total Assets CurrencyPair Assets : 3
Total Futures Assets : 2


## Using `filter_with_asset_class`

The `filter_with_asset_class` method is an extended version of the `filter` method.
It ensures that each returned asset is an instance of its correct class 

### When to Use?
- When querying multiple asset types and needing them returned with the correct class.
- When working with assets across execution venues and requiring proper type differentiation.

### Example 1: Query all asset types for a symbol


In [ ]:
# Query Binance Spot assets with correct asset classes
all_btc_assets = Asset.filter_with_asset_class(symbol="BTCUSDT")

# Displaying asset information
for asset in all_btc_assets:
    print(f"Asset: {asset.symbol},  Class: {type(asset).__name__}")


Asset: BTCUSDT,  Class: AssetCurrencyPair
Asset: BTCUSDT,  Class: AssetCurrencyPair
Asset: BTCUSDT,  Class: AssetFutureUSDM
Asset: BTCUSDT,  Class: AssetCurrencyPair
Asset: BTCUSDT,  Class: AssetFutureUSDM


### Example 2: Using FIGI Information to Narrow Down Results

Now imagine we only want currency pairs instead of futures.
 In this case, we can use FIGI details to identify the asset classes we are specifically interested in.



In [21]:
all_curreny_pairs_bitcoin= Asset.filter_with_asset_class(symbol="BTCUSDT",
figi_details__security_market_sector=MARKETS_CONSTANTS.FIGI_MARKET_SECTOR_CURNCY,
figi_details__security_type=MARKETS_CONSTANTS.FIGI_SECURITY_TYPE_CRYPTO
)

print(all_curreny_pairs_bitcoin)

[AssetCurrencyPair: 192403, AssetCurrencyPair: 192404, AssetCurrencyPair: 200468]
